In [6]:
%pip install fuzzywuzzy python-levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.2 MB/s eta 0:00:00


After installing the necessary libraries, please run the code cell again.

In [17]:
import pandas as pd
import chardet
from fuzzywuzzy import process, fuzz
import re
from tqdm import tqdm

# --- Langkah 1: Pemuatan dan Pembersihan Awal ---
files = ['lpdp_cs.csv', 'QS_2026_cleaned.csv', 'cwur2025_cleaned.csv', 'the2025_cleaned.csv']
dataframes = {}
for f in files:
    with open(f, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    dataframes[f.split('.')[0]] = pd.read_csv(f, encoding=encoding)

df_lpdp = dataframes['lpdp_cs']
df_qs = dataframes['QS_2026_cleaned']
df_cwur = dataframes['cwur2025_cleaned']
df_the = dataframes['the2025_cleaned']

def clean_rank(rank_series):
    return pd.to_numeric(rank_series.astype(str).str.extract(r'(\d+)')[0], errors='coerce')

df_qs['Rank'] = clean_rank(df_qs['Rank'])
df_cwur['World Rank'] = clean_rank(df_cwur['World Rank'])
df_the['Rank'] = clean_rank(df_the['Rank'])

# --- LANGKAH KUNCI: Pemisahan Nama Universitas & Negara pada Data THE ---
def split_uni_country(name_country):
    match = re.match(r'^(.*[a-z]|\s)(?=[A-Z][a-zA-Z\s]*$)', name_country)
    if match:
        return match.group(1).strip()
    return name_country.strip()

print("Memisahkan nama universitas dari negara di data THE...")
df_the['Cleaned Name'] = df_the['Name Country/Region'].apply(split_uni_country)
print("Pemisahan selesai.")


# --- Langkah 2: Fungsi Perburuan Peringkat ---
qs_dict = df_qs.set_index('Institution')['Rank'].to_dict()
cwur_dict = df_cwur.set_index('Institution')['World Rank'].to_dict()
the_dict = df_the.set_index('Cleaned Name')['Rank'].to_dict() # Menggunakan nama bersih

qs_names = list(qs_dict.keys())
cwur_names = list(cwur_dict.keys())
the_names = list(the_dict.keys()) # Menggunakan nama bersih

def hunt_for_rank(uni_name, name_list, rank_dict, threshold=88):
    if not name_list or not uni_name:
        return None
    best_match = process.extractOne(uni_name, name_list, scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= threshold:
        return rank_dict.get(best_match[0])
    return None

# --- Langkah 3: Eksekusi Perburuan ---
print("Memulai perburuan peringkat dengan metode terbaik...")

qs_ranks, cwur_ranks, the_ranks = [], [], []

for name in tqdm(df_lpdp['Name'], desc="Processing Universities"):
    qs_ranks.append(hunt_for_rank(name, qs_names, qs_dict))
    cwur_ranks.append(hunt_for_rank(name, cwur_names, cwur_dict))
    the_ranks.append(hunt_for_rank(name, the_names, the_dict))

df_lpdp['QS Rank 2026'] = qs_ranks
df_lpdp['CWUR Rank 2025'] = cwur_ranks
df_lpdp['THE Rank 2025'] = the_ranks
print("Perburuan peringkat selesai.")

# --- Langkah 4: Simpan Hasil Final Terbaik ---
df_lpdp.to_csv('merged_rankings_best_result.csv', index=False)

print("\nProses dengan metode terbaik selesai.")
print("Hasilnya disimpan dalam file 'merged_rankings_best_result.csv'.\n")
print("Pratinjau 30 baris pertama dari data yang paling akurat:")
print(df_lpdp.head(30))
print("\nInfo ringkas dari dataframe gabungan:")
df_lpdp.info()

Memisahkan nama universitas dari negara di data THE...
Pemisahan selesai.
Memulai perburuan peringkat dengan metode terbaik...


Processing Universities: 100%|██████████| 158/158 [00:12<00:00, 12.47it/s]

Perburuan peringkat selesai.

Proses dengan metode terbaik selesai.
Hasilnya disimpan dalam file 'merged_rankings_best_result.csv'.

Pratinjau 30 baris pertama dari data yang paling akurat:
    No                                               Name           Negara  \
0    1                               University of Oxford          Inggris   
1    2                                 Harvard University  Amerika Serikat   
2    3                                Stanford University  Amerika Serikat   
3    4        Massachusetts Institute of Technology (MIT)  Amerika Serikat   
4    5                            University of Cambridge          Inggris   
5    6       California Institute of Technology (Caltech)  Amerika Serikat   
6    7                              University of Chicago  Amerika Serikat   
7    8                                    Yale University  Amerika Serikat   
8    9                               Princeton University  Amerika Serikat   
9   10                        

In [22]:
import pandas as pd
import re

print("--- Memulai Proses Verifikasi dengan Pencatatan Ketidakcocokan ---")

# --- 1. Memuat Berkas ---
try:
    final_df = pd.read_csv('merged_rankings_best_result.csv')
    qs_raw = pd.read_csv('QS_2026_cleaned.csv')
    cwur_raw = pd.read_csv('cwur2025_cleaned.csv')
    the_raw = pd.read_csv('the2025_cleaned.csv')
    print("Semua berkas berhasil dimuat.")
except FileNotFoundError:
    print("Verifikasi Gagal: Pastikan file 'merged_rankings_best_result.csv' ada di direktori yang sama.")
    exit()

# --- 2. Siapkan Fungsi Bantuan & Wadah untuk Error ---
def clean_rank_value(rank_str):
    if pd.isna(rank_str): return None
    match = re.search(r'(\d+)', str(rank_str))
    if match: return float(match.group(1))
    return None

def verify_rank(university_name, df_raw, name_col, rank_col):
    row = df_raw[df_raw[name_col].str.contains(university_name.split('(')[0].strip(), na=False)]
    if not row.empty:
        original_rank = row.iloc[0][rank_col]
        return clean_rank_value(original_rank)
    return None

# Siapkan list kosong untuk menyimpan data ketidakcocokan
mismatches = []

# --- 3. Lakukan Verifikasi pada SEMUA Data (bukan hanya sampel) ---
print("\nMemverifikasi semua universitas di hasil akhir...")
print("-" * 50)

for index, row in tqdm(final_df.iterrows(), total=final_df.shape[0], desc="Verifying"):
    uni_name = row['Name']

    # Ambil peringkat dari hasil merge
    merged_qs = row['QS Rank 2026']
    merged_cwur = row['CWUR Rank 2025']
    merged_the = row['THE Rank 2025']

    # Cari peringkat pembanding dari data mentah
    original_qs = verify_rank(uni_name, qs_raw, 'Institution', 'Rank')
    original_cwur = verify_rank(uni_name, cwur_raw, 'Institution', 'World Rank')
    original_the = verify_rank(uni_name, the_raw, 'Name Country/Region', 'Rank')

    # Bandingkan dan catat jika ada perbedaan
    if not pd.isna(merged_qs) and merged_qs != original_qs:
        mismatches.append({
            'Universitas': uni_name,
            'Sumber Peringkat': 'QS',
            'Peringkat di Hasil Merge': merged_qs,
            'Peringkat di Data Asli': original_qs
        })

    if not pd.isna(merged_cwur) and merged_cwur != original_cwur:
        mismatches.append({
            'Universitas': uni_name,
            'Sumber Peringkat': 'CWUR',
            'Peringkat di Hasil Merge': merged_cwur,
            'Peringkat di Data Asli': original_cwur
        })

    if not pd.isna(merged_the) and merged_the != original_the:
        mismatches.append({
            'Universitas': uni_name,
            'Sumber Peringkat': 'THE',
            'Peringkat di Hasil Merge': merged_the,
            'Peringkat di Data Asli': original_the
        })

# --- 4. Simpan Hasil Ketidakcocokan ---
print("-" * 50)
if mismatches:
    mismatch_df = pd.DataFrame(mismatches)
    mismatch_df.to_csv('ketidakcocokan_verifikasi.csv', index=False)
    print(f"✅ Ditemukan {len(mismatch_df)} ketidakcocokan. Hasilnya telah disimpan di 'ketidakcocokan_verifikasi.csv'")
    print("\nBerikut adalah pratinjau data ketidakcocokan:")
    print(mismatch_df.head())
else:
    print("✅ Verifikasi Selesai. Tidak ditemukan ketidakcocokan sama sekali.")

--- Memulai Proses Verifikasi dengan Pencatatan Ketidakcocokan ---
Semua berkas berhasil dimuat.

Memverifikasi semua universitas di hasil akhir...
--------------------------------------------------


Verifying: 100%|██████████| 158/158 [00:00<00:00, 187.87it/s]

--------------------------------------------------
✅ Ditemukan 57 ketidakcocokan. Hasilnya telah disimpan di 'ketidakcocokan_verifikasi.csv'

Berikut adalah pratinjau data ketidakcocokan:
                                         Universitas Sumber Peringkat  \
0  ETH Zurich - Swiss Federal Institute of Techno...               QS   
1                    UCL (University College London)             CWUR   
2                   University of Michigan-Ann Arbor             CWUR   
3                        The University of Edinburgh               QS   
4                        The University of Edinburgh             CWUR   

   Peringkat di Hasil Merge  Peringkat di Data Asli  
0                       7.0                     NaN  
1                      20.0                     NaN  
2                      16.0                     NaN  
3                      34.0                     NaN  
4                      51.0                     NaN  


In [30]:
import pandas as pd
import re

print("--- Memulai Proses Penggabungan dengan Peringkat Computer Science ---")

# --- 1. Memuat Berkas yang Sudah Ada ---
try:
    # Memuat hasil utama yang sudah dikurasi dari sel sebelumnya
    df_main = pd.read_csv('merged_rankings_final_curated.csv')

    # Memuat data peringkat computer science
    df_cs_raw = pd.read_csv('thecomputerscience2025.csv')

    print("✅ Berkas utama dan berkas Computer Science berhasil dimuat.")

except FileNotFoundError as e:
    print(f"❌ Gagal: Pastikan berkas '{e.filename}' sudah ada sebelum menjalankan sel ini.")
    # Hentikan eksekusi jika ada file yang tidak ditemukan
    exit()

# --- 2. Membersihkan Berkas Computer Science (sesuai kebutuhan kita) ---
def clean_rank(rank_series):
    return pd.to_numeric(rank_series.astype(str).str.extract(r'(\d+)')[0], errors='coerce')

def split_uni_country(name_country):
    if not isinstance(name_country, str): return "", ""
    match = re.match(r'^(.*[a-z]|\s)(?=[A-Z][a-zA-Z\s]*$)', name_country)
    if match:
        return match.group(1).strip()
    return name_country.strip()

print("Membersihkan data peringkat Computer Science...")
df_cs_raw['Rank'] = clean_rank(df_cs_raw['Rank'])
df_cs_raw['Name'] = df_cs_raw['Name Country/Region'].apply(split_uni_country)
df_cs_cleaned = df_cs_raw[['Rank', 'Name']].copy()
df_cs_cleaned.rename(columns={'Rank': 'THE CS Rank 2025'}, inplace=True)
print("Pembersihan selesai.")

# --- 3. Melakukan Penggabungan (Merge) ---
print("\nMemulai proses penggabungan...")

# Gunakan 'left' merge untuk menjaga semua data utama, lalu tambahkan peringkat CS
df_merged_final = pd.merge(df_main, df_cs_cleaned, on='Name', how='left')

print("✅ Proses penggabungan selesai.")

# --- 4. Simpan dan Tampilkan Hasil Akhir ---
final_filename = 'final_merged_with_cs_ranking.csv'
df_merged_final.to_csv(final_filename, index=False)

print(f"\nHasil akhir penggabungan telah disimpan ke '{final_filename}'")
print("\nBerikut adalah pratinjau dari data final yang telah digabungkan:")
print(df_merged_final.head(30))

--- Memulai Proses Penggabungan dengan Peringkat Computer Science ---
✅ Berkas utama dan berkas Computer Science berhasil dimuat.
Membersihkan data peringkat Computer Science...
Pembersihan selesai.

Memulai proses penggabungan...
✅ Proses penggabungan selesai.

Hasil akhir penggabungan telah disimpan ke 'final_merged_with_cs_ranking.csv'

Berikut adalah pratinjau dari data final yang telah digabungkan:
                                                 Name  No           Negara  \
0                                University of Oxford   1          Inggris   
1                                  Harvard University   2  Amerika Serikat   
2                                 Stanford University   3  Amerika Serikat   
3         Massachusetts Institute of Technology (MIT)   4  Amerika Serikat   
4                             University of Cambridge   5          Inggris   
5        California Institute of Technology (Caltech)   6  Amerika Serikat   
6                               University of C

--- Memulai Proses Re-Verifikasi Final ---
✅ Semua berkas berhasil dimuat.

Memverifikasi ulang semua universitas di hasil akhir...
--------------------------------------------------


Re-Verifying: 100%|██████████| 158/158 [00:02<00:00, 72.86it/s]

--------------------------------------------------
✅ Proses Verifikasi Ulang Selesai. SEMPURNA! Tidak ada ketidakcocokan yang ditemukan.
